# Tarefa Aula 8 Engenharia de Dados Awari
## Rogerio Veiga Andrade - 08/11/2023

### Importar CSVs da pasta /exercicios/municipios-estados/csv/ e salvar como arquivos JSON no MinIO.
### Importar JSONs da pasta de /exercicios/municipios-estados/csv/ e salvar como arquivos CSV no MinIO.
### Importar CSVs da pasta de /exercicios/municipios-estados/csv/ e salvar como arquivo .parquet no MinIO.

In [1]:
import pyspark
import boto3

from io import StringIO 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

In [2]:
conf = pyspark.SparkConf()

# Criando uma sessão com o Spark que existe localmente(atualmente configurado junto com o JupyterLab)
conf.setMaster("local[1]") 
conf.set("spark.driver.host", "awari-jupyterlab") \
    .set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol") \
    .set("parquet.enable.summary-metadata", "false") \
    .set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false") \
    .set("spark.driver.port", "20020") \
    .set("spark.hadoop.fs.s3a.endpoint", 'awari-nginx:9000') \
    .set("spark.hadoop.fs.s3a.endpoint.region", 'sa-east-1') \
    .set("spark.hadoop.fs.s3a.access.key", '6DdjRBILhyMi2G2i') \
    .set("spark.hadoop.fs.s3a.secret.key", '9LSWJZT7f0i8FgjzflQQYrOq9tvUQrD1') \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .set("spark.hadoop.com.amazonaws.services.s3.enableV2", "true") \
    .set("spark.hadoop.fs.s3a.committer.staging.conflict-mode", "replace") \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.committer.name", "directory") \
    .set("spark.hadoop.fs.s3a.committer.staging.tmp.path", "/tmp/staging")

conf.setAppName('AwariAula08-S33')
sc = pyspark.SparkContext(conf=conf)

spark = SparkSession(sc)

In [3]:
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='6DdjRBILhyMi2G2i',
    aws_secret_access_key='9LSWJZT7f0i8FgjzflQQYrOq9tvUQrD1',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [5]:
municipios = spark.read.option("header", True).csv("./arquivos/municipios.csv")
estados = spark.read.option("header", True).csv("./arquivos/estados.csv")
municipios.show()
estados.show()

+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+
|codigo_ibge|               nome|latitude|longitude|capital|codigo_uf|siafi_id|ddd|     fuso_horario|
+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+
|    5200050|    Abadia de Goiás|-16.7573| -49.4412|      0|       52|    1050| 62|America/Sao_Paulo|
|    3100104|Abadia dos Dourados|-18.4831| -47.3916|      0|       31|    4001| 34|America/Sao_Paulo|
|    5200100|          Abadiânia| -16.197| -48.7057|      0|       52|    9201| 62|America/Sao_Paulo|
|    3100203|             Abaeté|-19.1551| -45.4444|      0|       31|    4003| 37|America/Sao_Paulo|
|    1500107|         Abaetetuba|-1.72183| -48.8788|      0|       15|    0401| 91|America/Sao_Paulo|
|    2300101|            Abaiara|-7.34588| -39.0416|      0|       23|    1301| 88|America/Sao_Paulo|
|    2900108|             Abaíra|-13.2488| -41.6619|      0|       29|    3301| 77

In [39]:
municipios.write.option("header", True).option("name", "municipios.csv").mode("overwrite").format("csv").csv("s3a://aula-08/municipios_csv")
estados.write.option("header", True).option("name", "estados.csv").mode("overwrite").format("csv").csv("s3a://aula-08/estados_csv")

In [40]:
municipios.write.option("header", True).option("name", "municipios.json").mode("overwrite").format('json').save("s3a://aula-08/municipios_json")
estados.write.option("header", True).option("name", "estados.json").mode("overwrite").format("json").csv("s3a://aula-08/estados_json")

In [41]:
municipios.write.mode("overwrite").save("s3a://aula-08/municipios_parquet")
estados.write.mode("overwrite").save("s3a://aula-08/estados_parquet")

In [43]:
df_tmp = spark.read.option("header", True).parquet("s3a://aula-08/municipios_parquet")
df_tmp.show()
df_tmp = spark.read.option("header", True).parquet("s3a://aula-08/estados_parquet")
df_tmp.show()

+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+
|codigo_ibge|               nome|latitude|longitude|capital|codigo_uf|siafi_id|ddd|     fuso_horario|
+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+
|    5200050|    Abadia de Goiás|-16.7573| -49.4412|      0|       52|    1050| 62|America/Sao_Paulo|
|    3100104|Abadia dos Dourados|-18.4831| -47.3916|      0|       31|    4001| 34|America/Sao_Paulo|
|    5200100|          Abadiânia| -16.197| -48.7057|      0|       52|    9201| 62|America/Sao_Paulo|
|    3100203|             Abaeté|-19.1551| -45.4444|      0|       31|    4003| 37|America/Sao_Paulo|
|    1500107|         Abaetetuba|-1.72183| -48.8788|      0|       15|    0401| 91|America/Sao_Paulo|
|    2300101|            Abaiara|-7.34588| -39.0416|      0|       23|    1301| 88|America/Sao_Paulo|
|    2900108|             Abaíra|-13.2488| -41.6619|      0|       29|    3301| 77